# Análisis NLP con Spacy: Harry Potter y la piedra filosofal (pelicula)

Por: [Leonardo Genzano](https://www.linkedin.com/in/leonardo-genzano-1b275193/)

Estaremos utilizando Spacy para hacer un análisis simple de NLP (Natural Language Processing) a la primer pelicula de Harry Potter.<br>
NLP se usa para muchisimas cosas, por ejemplo reconocimiento de texto, traducción automatica, generación de lenguaje,resumen de textos, deteccion de sentimientos, etc. <br>
<br>
Los datos de entrada contienen:<br>
* Character: personaje que está hablando
* Sentence: su diálogo

Consideraciones:
* los datos están en inglés
* para ver el código, presiona el botón Code que esta arriba a la derecha de cada visualización. Lo oculto para que no se haga largo el notebook
* voy a estar utilizando algunas funciones de Spacy, que es una biblioteca muy completa para hacer análisis NLP. https://spacy.io/usage
* Este notebook no fue realizado por un experto. Seguramente haya mejores maneras de afrontar las mismas problematicas. Quedo atento a cualquier comentario o sugerencia 

![The kaggle logo][1]
[1]: https://i.pinimg.com/originals/15/17/d7/1517d762a06a0a6b84f1b58b71b50a24.jpg

In [ ]:
pip install funpymodeling

In [ ]:
pip install matplotlib==3.1.3

In [ ]:
import pandas as pd
from funpymodeling.exploratory import freq_tbl, status, profiling_num, cat_vars, num_vars
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
import squarify
from IPython.display import YouTubeVideo

In [ ]:
import spacy
#import random
#from collections import Counter #for counting
plt.style.use('seaborn')
sns.set(font_scale=2)
#import json
#def pretty_print(pp_object):
  #  print(json.dumps(pp_object, indent=2))

In [ ]:
#!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')
# python -m spacy download en_vectors_web_lg

**************************

# Preparación de datos

In [ ]:
with open('../input/harry-potter-dataset/Harry Potter 1.csv', 'r', errors='ignore') as file:
    data = file.read()

In [ ]:
#convierto string a df
df = pd.DataFrame([x.split(';') for x in data.split('\n')])
#agrego headers y elimino primera fila
df= df.rename(columns={0 : "Character",  1: "Sentence" })
df= df.drop([0])
df = df.reset_index(drop=True)

In [ ]:
status(df)

In [ ]:
df[df["Sentence"].isnull()  ]

In [ ]:
df.loc[216,'Sentence'] = "All students must be equipped with one standard size 2 pewter cauldron and may bring, if they desire, either an owl, a cat or a toad."

In [ ]:
df= df.drop([217,615,1111,1590])
df = df.reset_index(drop=True)

In [ ]:
#Viendo los valores unicos de "Character", vemos que algunos strings estan "duplicados". 
# Esto sucede porque en algunos registros, el nombre del personaje aparece con espacios al final
# entonces lo solusionamos asi:
df["Character"]=df["Character"].str.strip()

In [ ]:
#df.to_csv('HP.csv', index=False, sep =';')

**********************

# Análisis NLP con Spacy

# General

In [ ]:
# Creamos objeto doc
text=df['Sentence'].str.cat(sep=' ')
doc = nlp(text)

****¿Qué personaje tiene más lineas?****

In [ ]:
freq_tbl(df["Character"]).head(10)

****¿Cuántas veces se nombra a la escuela Hogwarts?****

In [ ]:
data.count("Hogwarts")

****¿Cuál es la casa más nombrada? (a que ya saben el resultado..)****

In [ ]:
HP_houses = []
for token in doc:
    if  (token.text == "Slytherin" or token.text =="Ravenclaw" or token.text =="Gryffindor" or token.text =="Hufflepuff"):
        HP_houses.append(token)
        
HP_houses = [str(x) for x in HP_houses]


In [ ]:
df_houses = pd.DataFrame(HP_houses, columns=["HP_houses"])

In [ ]:
labels = ['Gryffindor','Slytherin','Hufflepuff','Ravenclaw']
sizes = df_houses.value_counts().values
colors = ['#A93226','#1E8449','#F1C40F','#1A5276']

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',colors=colors,pctdistance=0.85,
        shadow=True, startangle=90)

#draw internal circle
centre_circle = plt.Circle((0,0),0.6,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)



# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.show()

#https://medium.com/@kvnamipara/a-better-visualisation-of-pie-charts-by-matplotlib-935b7667d77f


In [ ]:
freq_tbl(HP_houses)

****Personas más nombradas****

In [ ]:
HP_persons = []
for token in doc:
    if  ( token.ent_type_ == "PERSON" ):
        HP_persons.append(token)
        
HP_persons = [str(x) for x in HP_persons]


In [ ]:
df_persons = pd.DataFrame(HP_persons, columns=["Personas"])
df_persons
plt.figure(figsize=(5,4))
sns.countplot(y="Personas",
             data=df_persons,
             order=df_persons["Personas"].value_counts().iloc[:10].index)
plt.title("Personas más nombradas")
plt.show()



****¿Cuáles son los sustantivos más utilizados?****

In [ ]:
hp_nouns = []
for token in doc:
    if (not token.is_stop) and (token.pos_ == "NOUN") and (len(str(token))>2):
        hp_nouns.append(token)
        
hp_nouns = [str(x) for x in hp_nouns]

In [ ]:
df_nouns = pd.DataFrame(hp_nouns, columns=["Sustantivos"])

In [ ]:
#filtro los datos para que el grafico tenga más sentido. 
data_treemap = df_nouns.groupby("Sustantivos").filter(lambda x: len(x) > 10)


# Prepare Data
dftreemap = data_treemap.groupby('Sustantivos').size().reset_index(name='counts')
labels = dftreemap.apply(lambda x: str(x[0]) + "\n (" + str(x[1]) + ")", axis=1)
sizes = dftreemap['counts'].values.tolist()
colors = [plt.cm.Spectral(i/float(len(labels))) for i in range(len(labels))]

# Draw Plot
plt.figure(figsize=(10,7), dpi= 80)
squarify.plot(sizes=sizes, label=labels, color=colors, alpha=.8)

# Decorate
plt.title('Sustantivos más utilizados')
plt.axis('off')
plt.show()

****¿Cuales son los adjetivos más utilizados?****

In [ ]:
HP_adjs = []
for token in doc:
    if (not token.is_stop) and (token.pos_ == "ADJ") and (len(str(token))>2):
        HP_adjs.append(token)
        
HP_adjs = [str(x) for x in HP_adjs]

In [ ]:
df_adjs = pd.DataFrame(HP_adjs, columns=["Adjetivos"])

In [ ]:
from wordcloud import WordCloud
words = [i for i in df_adjs["Adjetivos"]]
    
words = " ".join(words)

plt.subplots(figsize=(28,12))
wordcloud = WordCloud(
                          background_color='white',
    colormap="cividis",
                          width=2048,
                          height=1024
                          ).generate(words)
plt.imshow(wordcloud)
plt.axis('off')
#plt.savefig('graph.png')

plt.show()

# Análisis Harry Potter (personaje)

<img src="https://media1.tenor.com/images/2d7bfb9b3e6170a5a63605fb5c913e8f/tenor.gif?itemid=10629771">

In [ ]:
textHarry = df[df.Character == "Harry"]['Sentence'].str.cat(sep=' ')
docHarry = nlp(textHarry)

****¿Cuáles son las personas más nombradas por Harry?****

In [ ]:
harry_persons = []
for token in docHarry:
    if  ( token.ent_type_ == "PERSON" ):
        harry_persons.append(token)
        
harry_persons = [str(x) for x in harry_persons]

In [ ]:
df_persons_harry = pd.DataFrame(harry_persons, columns=["Personas"])
df_persons
plt.figure(figsize=(5,4))
sns.countplot(y="Personas",
             data=df_persons_harry,
             order=df_persons_harry["Personas"].value_counts().iloc[:10].index)
plt.title("Personas más nombradas")
plt.show()

****¿Cuales son los adjetivos más utilizados por Harry?****

In [ ]:
harry_adjs = []
for token in docHarry:
    if (not token.is_stop) and (token.pos_ == "ADJ")  and (len(str(token))>2):
        harry_adjs.append(token)
        
harry_adjs = [str(x) for x in harry_adjs]

In [ ]:
df_adjs_Harry = pd.DataFrame(harry_adjs, columns=["Adjetivos"])

In [ ]:
from wordcloud import WordCloud
words = [i for i in df_adjs_Harry["Adjetivos"]]
    
words = " ".join(words)

plt.subplots(figsize=(28,12))
wordcloud = WordCloud(
                          background_color='white',
    colormap="inferno",
                          width=2048,
                          height=1024
                          ).generate(words)
plt.imshow(wordcloud)
plt.axis('off')
#plt.savefig('graph.png')

plt.show()

# Análisis Draco Malfoy

<img src="https://i.pinimg.com/originals/38/e8/f2/38e8f2dfda8036deede516016df545bc.gif">



In [ ]:
textMalfoy = df[df.Character == "Malfoy"]['Sentence'].str.cat(sep=' ')

docMalfoy = nlp(textMalfoy)

****¿Cuáles son las personas más nombradas por Draco?****

In [ ]:
malfoy_persons = []
for token in docMalfoy:
    if  ( token.ent_type_ == "PERSON" ):
        malfoy_persons.append(token)
        
malfoy_persons = [str(x) for x in malfoy_persons]

In [ ]:
df_persons_malfoy = pd.DataFrame(malfoy_persons, columns=["Personas"])
df_persons
plt.figure(figsize=(5,4))
sns.countplot(y="Personas",
             data=df_persons_malfoy,
             order=df_persons_malfoy["Personas"].value_counts().iloc[:10].index)
plt.title("Personas más nombradas")
plt.show()

****¿Cuales son los adjetivos más utilizados por Draco Malfoy?****

In [ ]:
Malfoy_adjs = []
for token in docMalfoy:
    if (not token.is_stop) and (token.pos_ == "ADJ")  and (len(str(token))>2):
        Malfoy_adjs.append(token)
        
Malfoy_adjs = [str(x) for x in Malfoy_adjs]

In [ ]:
df_adjs_Malfoy = pd.DataFrame(Malfoy_adjs, columns=["Adjetivos"])

In [ ]:
from wordcloud import WordCloud
words = [i for i in df_adjs_Malfoy["Adjetivos"]]
    
words = " ".join(words)

plt.subplots(figsize=(28,12))
wordcloud = WordCloud(
                          background_color='white',
    colormap="viridis",
                          width=2048,
                          height=1024
                          ).generate(words)
plt.imshow(wordcloud)
plt.axis('off')
#plt.savefig('graph.png')

plt.show()

# Análisis Dumbledore

<img src="https://i.imgur.com/3a9el.gif?noredirect">

In [ ]:
textDumbledore = df[df.Character == "Dumbledore"]['Sentence'].str.cat(sep=' ')
docDumbledore = nlp(textDumbledore)

****¿Cuales son las personas más nombradas por Dumbledore?****

In [ ]:
Dumbledore_persons = []
for token in docDumbledore:
    if (token.ent_type_ == "PERSON") :
        Dumbledore_persons.append(token)
        
Dumbledore_persons = [str(x) for x in Dumbledore_persons]


In [ ]:
df_person_Dumbledore = pd.DataFrame(Dumbledore_persons, columns=["Personas"])
df_person_Dumbledore
plt.figure(figsize=(5,4))
sns.countplot(y="Personas",
             data=df_person_Dumbledore,
             order=df_person_Dumbledore["Personas"].value_counts().iloc[:10].index)
plt.show()

****¿Cuales son los adjetivos más utilizados por Dumbledore?****

In [ ]:
Dumbledore_adjs = []
for token in docDumbledore:
    if (not token.is_stop) and (token.pos_ == "ADJ")  and (len(str(token))>2):
        Dumbledore_adjs.append(token)
        
Dumbledore_adjs = [str(x) for x in Dumbledore_adjs]

In [ ]:
df_adjs_Dumbledore = pd.DataFrame(Dumbledore_adjs, columns=["Adjetivos"])

In [ ]:
from wordcloud import WordCloud
words = [i for i in df_adjs_Dumbledore["Adjetivos"]]
    
words = " ".join(words)

plt.subplots(figsize=(28,12))
wordcloud = WordCloud(
                          background_color='white',
    colormap="inferno",
                          width=2048,
                          height=1024
                          ).generate(words)
plt.imshow(wordcloud)
plt.axis('off')
#plt.savefig('graph.png')

plt.show()

# Análisis Severus Snape

<img src="https://media1.tenor.com/images/7806f69d9655adc2d541b3e6aa301912/tenor.gif?itemid=4874386">

In [ ]:
textSnape = df[df.Character == "Snape"]['Sentence'].str.cat(sep=' ')
docSnape = nlp(textSnape)

****¿Cuales son las personas más nombradas por Snape?****

In [ ]:
Snape_persons = []
for token in docSnape:
    if (token.ent_type_ == "PERSON") :
        Snape_persons.append(token)
        
Snape_persons = [str(x) for x in Snape_persons]


In [ ]:
df_persons_Snape = pd.DataFrame(Snape_persons, columns=["Personas"])
df_persons_Snape
plt.figure(figsize=(5,4))
sns.countplot(y="Personas",
             data=df_persons_Snape,
             order=df_persons_Snape["Personas"].value_counts().iloc[:10].index)
plt.show()

****¿Cuáles son los adjetivos mas usados por Snape? (Snape siendo Snape)****

In [ ]:
HP_adjs_snape = []
for token in docSnape:
    if (not token.is_stop) and (token.pos_ == "ADJ") and (len(str(token))>2):
        HP_adjs_snape.append(token)
        
HP_adjs_snape = [str(x) for x in HP_adjs_snape]

In [ ]:
df_adjs_snape = pd.DataFrame(HP_adjs_snape, columns=["Adjetivos"])

In [ ]:
from wordcloud import WordCloud
words = [i for i in df_adjs_snape["Adjetivos"]]
    
words = " ".join(words)

plt.subplots(figsize=(28,12))
wordcloud = WordCloud(
                          background_color='white',
    colormap="viridis",
                          width=2048,
                          height=1024
                          ).generate(words)
plt.imshow(wordcloud)
plt.axis('off')
#plt.savefig('graph.png')

plt.show()

In [ ]:
YouTubeVideo('2pCaH0hER9A', width=800, height=450)

# Proximos pasos:<br>
* Seguir con los análisis de otras peliculas de la saga
* Analizar varias peliculas a la vez (esto sería muy interesante)
* Detección de sentimiento

<img src="https://thumbs.gfycat.com/GrandImmaculateCreature-small.gif">